In [2]:
import pandas as pd
import time
import numpy as np
import pycutest

testes = [
"ARGLINA", "ARGLINB", "BA-L1SPLS", "BIGGS6", "BROWNAL", "COATING",
"FLETCHCR", "GAUSS2LS", "GENROSE", "HAHN1LS", "HEART6LS", "HILBERTB",
"HYDCAR6LS", "LANCZOS1LS", "LANCZOS2LS", "LRIJCNN1", "LUKSAN12LS",
"LUKSAN16LS", "OSBORNEA", "PALMER1C", "PALMER3C", "PENALTY2", "PENALTY3",
"QING", "ROSENBR", "STRTCHDV", "TESTQUAD", "THURBERLS", "TRIGON1",
"TOINTGOR"]


In [ ]:

def bfgs(function, tol = 1.0e-5, max_iter = 10000):
    f = pycutest.import_problem(function)
    xk = f.x0
    '''Dado um ponto inicial xk de uma função f, este método tenta encontrar um mínimo local aplicando o algoritmo BFGS 
    por no máximo 10.000 iterações, ou até que a norma-infinito do gradiente seja menor ou igual ao valor de tolerância 1.0e-5'''
    Hk = I = np.eye(len(xk)) # matriz identidade utilizada como a aproximação inicial para a inversa da Hessiana
    num_iter = 0
    num_aval_f = 0
    num_aval_grad = 0
    start = time.process_time()
    grad_k = f.grad(xk) # gradiente no ponto xk
    num_aval_grad += 1
    norm_k = np.linalg.norm(grad_k, ord = np.inf) # norma-infinito do gradiente
    while (num_iter < max_iter) and (norm_k > tol): 
        dk = Hk @ (-1*grad_k) # direção de descida dada pelo produto entre a matriz Hk e o gradiente no ponto xk
        alpha, aval_f = armijo(f, xk, grad_k, dk) # passo alpha calculado a partir das condições de Armijo
        num_aval_f += aval_f
        xk_new = xk + alpha*dk # valor do ponto é atualizado no sentido oposto ao gradiente
        # cálculo do delta x e da variação do gradiente, utilizados para atualização da matriz Hk:
        sk = xk_new - xk
        yk = f.grad(xk_new) - grad_k
        num_aval_grad += 1
        # se o produto interno skyk é positivo, atualiza-se a matriz Hk através da fórmula a seguir, caso contrário mantém-se a mesma aproximação para a próxima iteração
        if np.dot(sk, yk) > 0.0:
            Vk = I - np.outer(sk, yk)/np.dot(sk, yk)
            Hk = Vk @ Hk @ Vk.T + np.outer(sk, sk)/np.dot(sk, yk)
        xk = xk_new # atualização do ponto
        grad_k = f.grad(xk) # atualização do gradiente
        num_aval_grad += 1
        norm_k = np.linalg.norm(grad_k, ord = np.inf) # atualização da norma
        num_iter += 1
    valor = f.obj(xk) # valor da função no último ponto avaliado (possível mínimo local)
    num_aval_f += 1
    end = time.process_time()
    tempo = end - start # tempo de execução do algoritmo
    # Configuração do retorno
    output = {
        "num_aval_f": num_aval_f,
        "num_aval_grad": num_aval_grad,
        "tempo": tempo
    }
    
    # Retorno diferenciado em caso de falha de convergência
    if num_iter >= max_iter:
        output = {k: -v for k, v in output.items()}
    return output

def armijo(f, xk, grad_k, dk, alpha = 1, beta = 0.5, eta = 1.0e-4, max_iter = 100):
    num_iter = 0
    aval_f = 0
    a = eta*np.dot(grad_k, dk)
    c = f.obj(xk + alpha*dk)
    aval_f += 1
    b = f.obj(xk)
    aval_f += 1
    while (num_iter < max_iter) and (c > b + a*alpha): # fórmula da condição de armijo
        alpha = alpha*beta # atualização do passo
        c = f.obj(xk + alpha*dk) # atualização do parâmetro c
        aval_f += 1
        num_iter += 1
    return alpha, aval_f


In [ ]:
data1 = {i: bfgs(i) for i in testes}
df = pd.DataFrame(data1)
pd.DataFrame(data1).T

,num_aval_f,num_aval_grad,tempo
ARGLINA,4.0,3.0,0.061390
ARGLINB,-451088.0,-20001.0,-150.369434
BA-L1SPLS,996.0,163.0,0.047474
BIGGS6,92.0,83.0,0.009451
BROWNAL,92.0,65.0,0.298808
COATING,1595.0,743.0,1.872224
FLETCHCR,4935.0,963.0,602.569253
GAUSS2LS,123.0,45.0,0.089385
GENROSE,6772.0,2559.0,303.463703
HAHN1LS,-119018.0,-20001.0,-5.708753


In [28]:
z = df.T
z.to_csv(r"bfgs_tabela.csv",index=False)